In [ ]:
# Install required packages if needed
# !pip install opencv-python scikit-image scikit-learn plotly pandas ipywidgets matplotlib

In [2]:
!pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/a4/7d/f1c30a92854540bf789e9cd5dde7ef49bbe63f855b85a2e6b3db8135c591/opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/39.5 MB 2.8 MB/s eta 0:00:14
    --------------------------------------- 0.8/39.5 MB 8.5 MB/s eta 0:00:05
   -- ------------------------------------- 2.1/39.5 MB 16.6 MB/s eta 0:00:03
   --- ------------------------------------ 3.0/39.5 MB 15.9 MB/s eta 0:00:03
   ---- ----------------------------------- 4.5/39.5 MB 20.4 MB/s eta 0:00:02
   ------ --------------------------------- 6.1/39.5 MB 22.9 MB/s eta 0:00:02
   ------- -------------------------------- 7.3/39.5 MB 23.4 MB/s eta 0:00:02
   ------- -------------------------------- 7.6/39.5 MB 21.2 MB/s eta 0:00:02
   --------- ------------------------------ 9.5/39.5 MB 23.3 MB/s eta 0:00:02
   -----

In [3]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, HTML
import ipywidgets as widgets
from image_segmentation import *

# Import our image segmentation module
# import sys
# sys.path.append('path/to/image_segmentation')
from image_segmentation import *

c:\Users\Attca\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


## 1. Set Up Directories and Parameters

In [ ]:
# Define directories
# Update these paths as needed
input_dir = "C:/path/to/input/images"  # Directory containing original images
output_dir = "C:/path/to/output/segmented_images"  # Directory for segmented images and data

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Set the number of clusters for K-means
n_clusters = widgets.IntSlider(
    value=4,
    min=2,
    max=10,
    step=1,
    description='Number of clusters:',
    continuous_update=False
)
display(n_clusters)

## 2. Process Images and Generate Cluster Data

In [ ]:
# Process all images in the input directory
df, processed_images = process_image_folder(input_dir, output_dir, n_clusters=n_clusters.value)

# Display the data
if df is not None:
    print(f"Processed {len(processed_images)} images")
    display(df.head())
else:
    print("No data to display. Check input directory and image processing.")

## 3. Visualize Clusters in Sample Images

In [ ]:
# Display a sample of processed images
def display_sample_images(processed_images, num_samples=3):
    if not processed_images:
        print("No processed images to display.")
        return
    
    # Select a sample of images
    sample_size = min(num_samples, len(processed_images))
    sample_indices = np.linspace(0, len(processed_images)-1, sample_size).astype(int)
    samples = [processed_images[i] for i in sample_indices]
    
    # Create figure
    plt.figure(figsize=(15, 5*sample_size))
    for i, (filename, img) in enumerate(samples):
        plt.subplot(sample_size, 1, i+1)
        # Convert from BGR to RGB for display
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(f"Segmented: {filename}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Display some sample images
if processed_images:
    display_sample_images(processed_images, 3)
else:
    print("No processed images to display.")

## 4. Merge and Rename Clusters

In [ ]:
# Create UI for merging clusters
def create_merge_ui(df):
    if df is None:
        print("No data available for merging clusters.")
        return None, None, None
    
    # Get all percent columns (clusters)
    percent_columns = [col for col in df.columns if col.endswith('_Percent')]
    cluster_indices = [col.split('_')[1] for col in percent_columns]
    
    # Create multi-select widget for clusters to merge
    clusters_to_merge = widgets.SelectMultiple(
        options=cluster_indices,
        description='Select clusters to merge:',
        disabled=False
    )
    
    # Create text input for new cluster name
    new_cluster_name = widgets.Text(
        value='merged',
        placeholder='Enter new cluster name',
        description='New name:',
        disabled=False
    )
    
    # Create button to perform merge
    merge_button = widgets.Button(
        description='Merge Clusters',
        disabled=False,
        button_style='',
        tooltip='Click to merge selected clusters'
    )
    
    # Display widgets
    display(clusters_to_merge, new_cluster_name, merge_button)
    
    return clusters_to_merge, new_cluster_name, merge_button

# Execute if we have data
if df is not None:
    clusters_to_merge, new_cluster_name, merge_button = create_merge_ui(df)
    
    # Initialize variable to store merged DataFrame
    df_merged = None
    
    # Define what happens when the merge button is clicked
    def on_merge_button_clicked(b):
        global df_merged
        if not clusters_to_merge.value:
            print("Please select at least one cluster to merge.")
            return
        
        # Merge selected clusters
        df_merged = merge_clusters(df, clusters_to_merge.value, new_cluster_name.value)
        
        # Display result
        print(f"Merged {len(clusters_to_merge.value)} clusters into '{new_cluster_name.value}'")
        display(df_merged.head())
        
        # Save merged data to CSV
        merged_csv_path = os.path.join(output_dir, 'cluster_data_merged.csv')
        df_merged.to_csv(merged_csv_path, index=False)
        print(f"Merged data saved to {merged_csv_path}")
    
    # Attach the function to the button
    merge_button.on_click(on_merge_button_clicked)

## 5. Generate Stacked Bar Chart of Cluster Coverage Over Time

In [ ]:
# Generate stacked bar chart
def create_stacked_bar_chart(dataframe, use_merged=False):
    if dataframe is None:
        print("No data available for generating chart.")
        return
    
    # Use either the original or merged DataFrame
    plot_df = df_merged if use_merged and 'df_merged' in globals() and df_merged is not None else dataframe
    
    # Sort by datetime if it's a datetime column
    if 'datetime' in plot_df.columns and pd.api.types.is_datetime64_any_dtype(plot_df['datetime']):
        plot_df = plot_df.sort_values('datetime')
    
    # Get columns for the y-axis (all percent columns)
    y_columns = [col for col in plot_df.columns if col.endswith('_Percent')]
    
    # Create chart
    fig = generate_stacked_bar_chart(plot_df, 'datetime', y_columns)
    fig.show()

# Create buttons for generating charts
if df is not None:
    # Button for original data
    original_chart_button = widgets.Button(
        description='Generate Chart (Original)',
        disabled=False,
        button_style='',
        tooltip='Generate chart with original clusters'
    )
    
    # Button for merged data
    merged_chart_button = widgets.Button(
        description='Generate Chart (Merged)',
        disabled=False,
        button_style='',
        tooltip='Generate chart with merged clusters'
    )
    
    # Display buttons
    display(widgets.HBox([original_chart_button, merged_chart_button]))
    
    # Define button click handlers
    def on_original_chart_button_clicked(b):
        create_stacked_bar_chart(df, use_merged=False)
    
    def on_merged_chart_button_clicked(b):
        create_stacked_bar_chart(df, use_merged=True)
    
    # Attach handlers
    original_chart_button.on_click(on_original_chart_button_clicked)
    merged_chart_button.on_click(on_merged_chart_button_clicked)

## 6. Export Results

In [ ]:
# Function to export the final data and save the chart
def export_results(dataframe, use_merged=False, chart_filename='cluster_coverage_chart.html'):
    if dataframe is None:
        print("No data available for export.")
        return
    
    # Use either the original or merged DataFrame
    export_df = df_merged if use_merged and 'df_merged' in globals() and df_merged is not None else dataframe
    
    # Export DataFrame to CSV
    export_csv_path = os.path.join(output_dir, 'final_cluster_data.csv')
    export_df.to_csv(export_csv_path, index=False)
    print(f"Final data exported to {export_csv_path}")
    
    # Generate and save chart
    y_columns = [col for col in export_df.columns if col.endswith('_Percent')]
    fig = generate_stacked_bar_chart(export_df, 'datetime', y_columns)
    chart_path = os.path.join(output_dir, chart_filename)
    fig.write_html(chart_path)
    print(f"Chart saved to {chart_path}")

# Create export button
if df is not None:
    export_button = widgets.Button(
        description='Export Final Results',
        disabled=False,
        button_style='success',
        tooltip='Export data and chart'
    )
    
    use_merged_checkbox = widgets.Checkbox(
        value=False,
        description='Use merged data',
        disabled=False
    )
    
    display(widgets.VBox([use_merged_checkbox, export_button]))
    
    def on_export_button_clicked(b):
        export_results(df, use_merged=use_merged_checkbox.value)
    
    export_button.on_click(on_export_button_clicked)